# Traffic Sign Identification using Deep Neural Networks

## Description
Lorem Ipsum

## Setup and Image Preprocessing

### Imports

In [10]:
import cv2
import pickle
import csv

#A tua mina, tem ganda vagina
#Mas não foste tu quem escavou bro, foi o Chico da Tina

### Preprocessing Function
Resizes and Normalizes all pictures in all of our datasets and then stores them in a dictionary.

In [11]:
def preprocess_images(file_name):
    path = './Dataset/'   # Change this for the path corresponding to your base Dataset directory

    img_infos = {'labels':[],'images':[]}

    with open(path+file_name,"r") as csv_file:
        reader = csv.reader(csv_file, delimiter=',', quotechar='|')
        next(reader)
        
        for row in reader:

            img = cv2.imread(path+row[-1], cv2.IMREAD_UNCHANGED)    
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #Convert image to Grayscale
            
            img = cv2.resize(img, (32,32),interpolation = cv2.INTER_AREA) #Resize the image to 32x32

            #img = cv2.addWeighted( img, 1.2, img, 0, 0) #Increase contrast
            
            cv2.normalize(img, img, 0, 255, cv2.NORM_MINMAX) #Normalize
            
            img_infos['images'].append(img) #Save Image pixels
            img_infos['labels'].append(row[-2]) #Save image Label

    return img_infos

### Save treated data as Pickle
Srsly, funniest thing I've ever seen

In [12]:
img_infos = preprocess_images("Test.csv")
with open("./Dataset/PickledData/Test.p", 'wb') as pickle_rick:
    pickle.dump(img_infos,pickle_rick)
    
img_infos = preprocess_images("Train.csv")
with open("./Dataset/PickledData/Train.p", 'wb') as pickle_rick:
    pickle.dump(img_infos,pickle_rick)

## Neural Network

### Step 0 - Imports

In [13]:
import random
import numpy as np
import pickle

import tensorflow as tf
from tensorflow import keras

from sklearn.utils import shuffle

### Step 1 - Get the data into the sets

In [14]:
with open("./Dataset/PickledData/Test.p", "rb") as f:
    test = pickle.load(f)
with open("./Dataset/PickledData/Train.p", "rb") as f:
    train = pickle.load(f)

# Get the training set
x_train, y_train = train['images'], train['labels']

# Get the cross validation set
x_cv = []
y_cv = []
for i in range(int(0.2*len(x_train))): # Randomly move 20% of the training examples into the cross-validation set
    index = random.randint(0, len(x_train)-1)
    x_cv.append(x_train.pop(index))
    y_cv.append(y_train.pop(index))
    
# Get the test set
x_test, y_test = test['images'], test['labels']

# Convert all our sets into numPy Arrays
x_train = np.array(x_train)
y_train = keras.utils.to_categorical(np.array(y_train))

x_cv = np.array(x_cv)
y_cv = keras.utils.to_categorical(np.array(y_cv))

x_test = np.array(x_test)
y_test = keras.utils.to_categorical(np.array(y_test))

### Step 2 - Treat the data (Normalize and Shuffle)

In [15]:
# Normalize the data sets (Subtract and divide by half of the range of values. Gets the values between -1 and 1)
x_train = ((x_train - 127.5) / 127.5)
x_cv = ((x_cv - 127.5) / 127.5)
x_test = ((x_test - 127.5) / 127.5)

# Shuffle the training set
x_train, y_train = shuffle(x_train, y_train)

### Step 3 - Define constants

In [16]:
EPOCHS = 50
BATCH_SIZE = 200

LEARNING_RATE = 0.002 # Define the learning rate to be used by our optimizer


### Step 4 - Define the Neural Network

In [17]:
def LeNetModel(no_output=6, rgb=True):
    input_shape = (32,32,3) if rgb else (32,32,1) # Define the image's size
    
    model = keras.Sequential() # Define the Keras Model
    
    # First Layer - Convolutional ; Input = 32x32x3 for RGB , 32x32x1 for GrayScale ; Output = 28x28x6
    model.add(keras.layers.Conv2D(
                filters=no_output, kernel_size=(3, 3),
                activation='relu', input_shape=input_shape
                )
             )
    
    # Second Layer - Subsampling (Average Pooling) ; Input = 28x28x6 ; Output = 14x14x6
    model.add(keras.layers.AveragePooling2D())
    
    # Third Layer - Convolutional ; Input = 14x14x6 ; Output = 10x10x16
    model.add(keras.layers.Conv2D(
                    filters=16, kernel_size=(3, 3),
                    activation='relu'
                    )
                 )
    
    # Fourth Layer - Subsampling (Average Pooling) ; Input = 10x10x16 ; Output = 5x5x16
    model.add(keras.layers.AveragePooling2D())
    
    model.add(keras.layers.Flatten()) # Flatten the last layer's output to pass it to the Fully Connected layers
    
    # Fifth Layer - Fully Connected ; Input = 5x5x16 ; Output = 120x1
    model.add(keras.layers.Dense(units=120, activation='relu'))

    # Sixth Layer - Fully Connected ; Input = 5x5x16 ; Output = 120x1
    model.add(keras.layers.Dense(units=84, activation='relu'))
    
    # Output Layer - Output = 43x1
    model.add(keras.layers.Dense(units=43, activation = 'softmax'))
    
    return model

### Step 5 - Create the model and define evaluation

In [19]:
model = LeNetModel(rgb=False)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              metrics=['accuracy'])

### Step 6 - Insert the training set

In [20]:
model.fit(x_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          verbose=1,
          validation_data=(x_cv, y_cv))

ValueError: Error when checking input: expected conv2d_2_input to have 4 dimensions, but got array with shape (31368, 32, 32)

In [9]:
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

12630/12630 [==============================] - 1s 105us/sample - loss: 0.9067 - accuracy: 0.9033
Test loss: 0.9067454308839935
Test accuracy: 0.90332544
